# Motivation

- What is your dataset?

- Why did you choose this/these particular dataset(s)?

- What was your goal for the end user's experience?

Overall our dataset consists of XXXX amount of theorists wikipedia pages, gathered from five wikipedia lists each representing a science with the overall theme of social science. By using the researchers references to each other we construct edges between the researchers, and therefore are able the map the overall science community of the social sciences. 

By looking into the researchers from each discipline we hope to find patterns in the connections both within the five categories but also between the five social sciences. We will first seek to answer the question of, what characteristics does the community of the five social sciences have? We will examine this question by looking into the over network properties of each followed by a network analysis. Thereafter we will examine the communities of the social sciences, by doing this we will answer these questions: Are the traditional categories within the social science community actually also apt/suitable for the social science community? Or are the categorizations just a historic relic which we should reject, and then embrace some other maybe more relevant categorizations within the social sciences? By looking into this it becomes possible to either deny or affirm the different scientists relevance within the different disciplines of the social science community. 

Our overall goal for the end user experience is to give a easy overview of the social sciences. We hope that our community detection models and our HSMB models maybe can give interested people a nuance in their view of the social sciences, instead of keeping to the old disciplines. By looking into our website interested researchers will maybe get inspiration to move out and beyond their own social science. 

# Basic

- Write about your choices in data cleaning and preprocessing

- Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)

### Extracting data from wikipedias API

To construct this "community" of social scientists, we found inspiration by examining the department for Social Science at Copenhagen University. The Social Science department have five different sciences: Sociology, Psychology, Economy, Anthropology, and Political Science. For each of these five categories we have a list of scientists on wikipedia. We used these five lists of scientists to find and categorise all the scientists in our final dataset. After gathering all the names from the five lists, we then gather each individual scientists wikipage. Thus our data gathering process has two parts: 1) get social scientists from each of the five lists, 2) gather each scientists wikipage. 

If a scientist is mentioned on more than one list we calssiffy them as being in a sixth category 'Multiple'. 

Below we go more in depth with each of the two steps and our choices in the data cleaning process. 

#### Get social scientists from each of the five lists

Here we called the wiki API for each of the five list and then sorted the data. When cleaning the lists for theorists we used a regular expression to: XXXXXXXXXXXXXXXXX (ESBEN KOM OG FORKLAR) 

In [ ]:
####################################CALLING THE API for the five lists####################################################
import requests
import pandas as pd
from bs4 import BeautifulSoup
import regex as re 
import json

def get_wiki(_page, get_txt = False):
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    title = f"titles={_page}"
    content = "prop=revisions&rvprop=content&rvslots=*"
    dataformat ="format=json"
    
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    print(query)
    
    if get_txt == True:
        resp = requests.get(query).json()
        page_id = [i for i in resp['query']['pages'].keys()][0] # get page id
        txt = resp['query']['pages'][page_id]['revisions'][0]['slots']['main']['*']
        return txt
    
    else:
        return requests.get(query).json()

resp_soc = get_wiki("List_of_sociologists")
resp_ant = get_wiki("List_of_anthropologists")
resp_pol = get_wiki("List_of_political_scientists")
resp_psy = get_wiki("List_of_psychologists")
resp_eco = get_wiki("List_of_economists")

In [ ]:
##################################Cleaning each social scientist discipline###################################
''' Soc '''
txt = resp_soc['query']['pages']['254243']['revisions'][0]['slots']['main']['*']
txt = txt.split('==A==')[1] # drop intro tekst
txt = txt.split('==References==')[0] # drop trailing stuff
sociologists = re.findall('\n\*(?: )?\[\[(.*?)(?:\|.*?)?\]\]*',txt)
print(f'Sociologists: {len(sociologists)}')

''' Ant '''
txt = resp_ant['query']['pages']['728']['revisions'][0]['slots']['main']['*']
txt = txt.split('==A==')[1] # drop intro tekst
txt = txt.split('==Fictional anthropologists==')[0] # drop trailing stuff
anthropologists = re.findall('\n\*(?: )?\[\[(.*?)(?:\|.*?)?\]\]*',txt)
print(f'Anthropologist: {len(anthropologists)}')

''' Eco '''
txt = resp_eco["query"]['pages']['10231']['revisions'][0]['slots']['main']['*']
txt = txt.split('==A==')[1] # drop intro tekst
txt = txt.split('==See also==')[0] # drop trailing stuff
economists = re.findall('\n\*(?: )?\[\[(.*?)(?:\|.*?)?\]\]*',txt)
print(f'economists: {len(economists)}')

''' Psy '''
txt = resp_psy['query']['pages']['199877']['revisions'][0]['slots']['main']['*']
txt = txt.split('== A ==')[1] # drop intro tekst
txt = txt.split('==See also==')[0] # drop trailing stuff
psychologists = re.findall('\n\*(?: )?\[\[(.*?)(?:\|.*?)?\]\]*',txt)
print(f'Psychologists: {len(psychologists)}')

''' Pol '''
txt = resp_pol['query']['pages']['37559']['revisions'][0]['slots']['main']['*']
txt = txt.split('== A ==')[1] # drop intro tekst
txt = txt.split('== See also ==')[0] # drop trailing stuff
political_scientists = re.findall('\n\*(?: )?\[\[(.*?)(?:\|.*?)?\]\]*',txt)
print(f'Political_scientists: {len(political_scientists)}')

# To dict
science_dict = {'soc':sociologists,
                'anth':anthropologists,
                'eco': economists,
                'psy': psychologists,
                'pol': political_scientists}

with open('science_name_dict.json', 'w', encoding = 'utf-8') as f:
    json.dump(science_dict,f)

#### Gather each scientists wikipage

For gathering the theorists wikipages we use the "get_wiki" function, which we also used to gather the fives lists before. But when gatherings XXXX amount of links instead of five it is also natural to meet some more problems. We both had the problem that some theorists pages redirected us to another page, which we need to call  

In [ ]:
import time
import copy

with open('science_name_dict.json', 'r', encoding = 'utf-8') as f:
    science_dict = json.load(f)

# Copy dict - deep copy as the values are nested in a list
new_science_dict = copy.deepcopy(science_dict)

    
''' Scrape the pages '''
for field, nodes in science_dict.items():
    for node in nodes:
        time.sleep(0.2)
        # A very few times it shows uses either a # or ?, and this always redirects to topic, 
        # We therefore skip it. An example is: 'Karen_Horney#Ten_neurotic_needs'
        if '#' in node or '?' in node: 
            new_science_dict[field].remove(node) # remove node 
            print('Skipping: ', node)
            continue
            
        node = node.replace(' ', '_')
        try:
            txt = get_wiki(node, get_txt = True)
            
            # Sometimes the page link we have gotten from one of our five lists is a "redirect"
            # Here it is nessecary to extract this new one. 
            if "#REDIRECT" in txt: # if it is a redirect
                new_science_dict[field].remove(node.replace('_', ' '))  # delete the old value 
                node = [re.findall('\[\[(.*?)(?:\|.*?)?\]\]', txt)[0]] # redirected link 
                print(f"## Redirect! {node}")
                new_science_dict[field] += node # append the new node (only element in a list) to the science_dict 
                node = node[0].replace(' ', '_') # extract from the list and replace 
                
                if '#' in node or '?' in node: 
                    new_science_dict[field].remove(node.replace('_', ' ')) # remove from dict
                    print('Skipping: ', node)
                    continue                    
                    
                txt = get_wiki(node, get_txt=True)
                
        except KeyError as e: # if the page is incomplete (red hyperlinks)
            new_science_dict[field].remove(node.replace('_', ' ')) # remove from dict 
            print(e, node)
            continue
        
        with open(f'wiki_content/{node}.txt', 'w', encoding = 'utf-8') as f:
            f.write(txt)
            
# Gem den opdaterede dict 
with open('science_name_dict.json', 'w', encoding = 'utf-8') as f:
    json.dump(new_science_dict,f)

#### Parsing the data for constructing the network

In [ ]:
for field, nodes in new_science_dict.items():
    for node in nodes:
        node = node.replace(' ','_')
        with open(f'wiki_content/{node}.txt', 'r', encoding = 'utf-8') as f:
            f.read()

#### Parsing the data for text analysis

# Tools, theory and analysis. Describe the process of theory to insight

- Talk about how you've worked with text, including regular expressions, unicode, etc.
Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.

- How did you use the tools to understand your dataset?

#### ANALYSE STRUKTUR: 
Explain the overall idea

- Analysis step 1
    - explain what you're interested in
    - explain the tool
    - apply the tool
    - discuss the outcome

- Analysis step 2
    - explain what you're interested in
    - explain the tool
    - apply the tool
    - discuss the outcome

- Analysis step 3,
... and so on until the analysis is done

# Discussion. Think critically about your creation

- What went well?,
- What is still missing? What could be improved?, Why?

# Constributions

- You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That’s what you should explain).

- It is not OK simply to write "All group members contributed equally".